In [130]:
import pandas as pd
csv_path= '/home/jupyter-nafisha/X-ray/CSVs/chexpert_orig.csv'
data= pd.read_csv(csv_path)

In [131]:
import pandas as pd

cols = [
    'No Finding',
    'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
    'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
    'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture'
]

# Safely convert to numeric
numeric_cols = (
    data[cols]
    .apply(pd.to_numeric, errors='coerce')
)

# Mask: no column has value 1
mask = ~numeric_cols.eq(1).any(axis=1)

df_no_positive_labels = data[mask].reset_index(drop=True)

df_no_positive_labels


,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00011/study13...,Female,22,Frontal,PA,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN
1,CheXpert-v1.0-small/train/patient00011/study13...,Female,22,Lateral,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN
2,CheXpert-v1.0-small/train/patient00012/study2/...,Female,55,Frontal,PA,NaN,-1.0,-1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0
3,CheXpert-v1.0-small/train/patient00012/study2/...,Female,55,Lateral,NaN,NaN,-1.0,-1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0
4,CheXpert-v1.0-small/train/patient00012/study1/...,Female,55,Frontal,PA,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17509,CheXpert-v1.0-small/train/patient64514/study1/...,Male,72,Frontal,AP,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
17510,CheXpert-v1.0-small/train/patient64523/study1/...,Female,90,Frontal,AP,NaN,NaN,NaN,0.0,NaN,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17511,CheXpert-v1.0-small/train/patient64533/study2/...,Male,75,Frontal,AP,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN
17512,CheXpert-v1.0-small/train/patient64537/study1/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,-1.0,NaN,-1.0,NaN,NaN,NaN


In [125]:
# df_no_positive_labels[df_no_positive_labels['Frontal/Lateral']=='Lateral']

### to check if there is any such case of no label in the version 2 training

In [126]:
# to check if there is any such case of no label in the version 2 training

import os
df_no_positive_labels['Path'] = df_no_positive_labels['Path'].str.replace('CheXpert-v1.0-small/train/', '')
df_no_positive_labels['Path'] = [os.path.join('chexpert_PA/', path) for path in df_no_positive_labels['Path']]
sec_csv= pd.read_csv('/home/jupyter-nafisha/X-ray/CSVs/train_combined.csv')
merged= sec_csv.merge(
    df_no_positive_labels,
    # data,
    left_on = 'image_id',
    right_on = 'Path',
    how= 'inner'
)

merged

,image_id,class_name,label,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,...,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices


### remove all files from the original dataset

In [132]:
# Filter rows whose path is NOT in df_no_positive_labels
df_filtered = data[
    ~data['Path'].isin(df_no_positive_labels['Path'])
].reset_index(drop=True)

In [133]:
df_filtered

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205895,CheXpert-v1.0-small/train/patient64536/study2/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
205896,CheXpert-v1.0-small/train/patient64536/study1/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
205897,CheXpert-v1.0-small/train/patient64537/study2/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1.0,0.0,1.0,NaN,NaN,NaN
205898,CheXpert-v1.0-small/train/patient64539/study1/...,Female,0,Frontal,AP,NaN,NaN,1.0,1.0,NaN,NaN,NaN,-1.0,1.0,0.0,NaN,NaN,NaN,0.0


### lateral analysis

In [134]:
df_lateral= df_filtered[df_filtered['Frontal/Lateral']=='Lateral']
df_lateral

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
6,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Lateral,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
8,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Lateral,NaN,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0
17,CheXpert-v1.0-small/train/patient00009/study1/...,Male,76,Lateral,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
19,CheXpert-v1.0-small/train/patient00010/study1/...,Female,50,Lateral,NaN,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202076,CheXpert-v1.0-small/train/patient60704/study1/...,Male,67,Lateral,NaN,NaN,NaN,-1.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
202079,CheXpert-v1.0-small/train/patient60706/study1/...,Female,90,Lateral,NaN,NaN,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,1.0,NaN,1.0
202081,CheXpert-v1.0-small/train/patient60707/study1/...,Female,38,Lateral,NaN,NaN,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0
202083,CheXpert-v1.0-small/train/patient60708/study1/...,Female,78,Lateral,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN


In [136]:
len(df_lateral[df_lateral['Sex']=='Male']), len(df_lateral[df_lateral['Sex']=='Female'])

(17666, 10251)

In [137]:
# Normal
lateral_normal = df_lateral[df_lateral['No Finding']==1]

# Abnormal
lateral_abnormal = df_lateral[df_lateral['No Finding']!=1]

In [161]:
lateral_normal_female = lateral_normal[lateral_normal['Sex']=='Female']
lateral_normal_male = lateral_normal[lateral_normal['Sex']=='Male']

lateral_abnormal_female = lateral_abnormal[lateral_abnormal['Sex']=='Female']
lateral_abnormal_male = lateral_abnormal[lateral_abnormal['Sex']=='Male']

In [162]:
RANDOM_STATE = 42

# Shuffle the dataframe
lateral_female_normal_shuffled = lateral_normal_female.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 1394
val_size = 298
test_size = 298

# Create splits
lateral_female_normal_train = lateral_female_normal_shuffled.iloc[:train_size]
lateral_female_normal_val   = lateral_female_normal_shuffled.iloc[train_size:train_size + val_size]
lateral_female_normal_test  = lateral_female_normal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [163]:
RANDOM_STATE = 42

# Shuffle the dataframe
lateral_male_normal_shuffled = lateral_normal_male.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 1400
val_size = 300
test_size = 300

# Create splits
lateral_male_normal_train = lateral_male_normal_shuffled.iloc[:train_size]
lateral_male_normal_val   = lateral_male_normal_shuffled.iloc[train_size:train_size + val_size]
lateral_male_normal_test  = lateral_male_normal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [164]:
RANDOM_STATE = 42

# Shuffle the dataframe
lateral_female_abnormal_shuffled = lateral_abnormal_female.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 1400
val_size = 300
test_size = 300

# Create splits
lateral_female_abnormal_train = lateral_female_abnormal_shuffled.iloc[:train_size]
lateral_female_abnormal_val   = lateral_female_abnormal_shuffled.iloc[train_size:train_size + val_size]
lateral_female_abnormal_test  = lateral_female_abnormal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [165]:
RANDOM_STATE = 42

# Shuffle the dataframe
lateral_male_abnormal_shuffled = lateral_abnormal_male.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 1400
val_size = 300
test_size = 300

# Create splits
lateral_male_abnormal_train = lateral_male_abnormal_shuffled.iloc[:train_size]
lateral_male_abnormal_val   = lateral_male_abnormal_shuffled.iloc[train_size:train_size + val_size]
lateral_male_abnormal_test  = lateral_male_abnormal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [169]:
lateral_val = pd.concat(
    [
        lateral_female_normal_val,
        lateral_female_abnormal_val,
        lateral_male_normal_val,
        lateral_male_abnormal_val
    ],
    axis=0,
    ignore_index=True
)

In [170]:
lateral_test = pd.concat(
    [
        lateral_female_normal_test,
        lateral_female_abnormal_test,
        lateral_male_normal_test,
        lateral_male_abnormal_test
    ],
    axis=0,
    ignore_index=True
)

In [171]:
lateral_train = pd.concat(
    [
        lateral_female_normal_train,
        lateral_female_abnormal_train,
        lateral_male_normal_train,
        lateral_male_abnormal_train
    ],
    axis=0,
    ignore_index=True
)

### frontal AP analysis

In [195]:
df_frontal = df_filtered[df_filtered['Frontal/Lateral']=='Frontal']
df_AP = df_frontal[df_frontal['AP/PA']=='AP']
df_AP

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
9,CheXpert-v1.0-small/train/patient00005/study2/...,Male,33,Frontal,AP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205895,CheXpert-v1.0-small/train/patient64536/study2/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
205896,CheXpert-v1.0-small/train/patient64536/study1/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
205897,CheXpert-v1.0-small/train/patient64537/study2/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1.0,0.0,1.0,NaN,NaN,NaN
205898,CheXpert-v1.0-small/train/patient64539/study1/...,Female,0,Frontal,AP,NaN,NaN,1.0,1.0,NaN,NaN,NaN,-1.0,1.0,0.0,NaN,NaN,NaN,0.0


In [196]:
len(df_AP[df_AP['Sex']=='Male']), len(df_AP[df_AP['Sex']=='Female'])

(88084, 64555)

In [197]:
# Normal
AP_normal = df_AP[df_AP['No Finding']==1]

# Abnormal
AP_abnormal = df_AP[df_AP['No Finding']!=1]

In [198]:
AP_normal_female = AP_normal[AP_normal['Sex']=='Female']
AP_normal_male = AP_normal[AP_normal['Sex']=='Male']

AP_abnormal_female = AP_abnormal[AP_abnormal['Sex']=='Female']
AP_abnormal_male = AP_abnormal[AP_abnormal['Sex']=='Male']

In [199]:
RANDOM_STATE = 42

# Shuffle the dataframe
AP_female_normal_shuffled = AP_normal_female.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 3706
val_size = 794
test_size = 794

# Create splits
AP_female_normal_train = AP_female_normal_shuffled.iloc[:train_size]
AP_female_normal_val   = AP_female_normal_shuffled.iloc[train_size:train_size + val_size]
AP_female_normal_test  = AP_female_normal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [200]:
RANDOM_STATE = 42

# Shuffle the dataframe
AP_female_abnormal_shuffled = AP_abnormal_female.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 3710
val_size = 795
test_size = 795

# Creabate splits
AP_female_abnormal_train = AP_female_abnormal_shuffled.iloc[:train_size]
AP_female_abnormal_val   = AP_female_abnormal_shuffled.iloc[train_size:train_size + val_size]
AP_female_abnormal_test  = AP_female_abnormal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [201]:
RANDOM_STATE = 42

# Shuffle the dataframe
AP_male_normal_shuffled = AP_normal_male.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 3710
val_size = 795
test_size = 795

# Cabate splits
AP_male_normal_train = AP_male_normal_shuffled.iloc[:train_size]
AP_male_normal_val   = AP_male_normal_shuffled.iloc[train_size:train_size + val_size]
AP_male_normal_test  = AP_male_normal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [202]:
RANDOM_STATE = 42

# Shuffle the dataframe
AP_male_abnormal_shuffled = AP_abnormal_male.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 3710
val_size = 795
test_size = 795

# Creabate splits
AP_male_abnormal_train = AP_male_abnormal_shuffled.iloc[:train_size]
AP_male_abnormal_val   = AP_male_abnormal_shuffled.iloc[train_size:train_size + val_size]
AP_male_abnormal_test  = AP_male_abnormal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [203]:
AP_train = pd.concat(
    [
        AP_female_normal_train,
        AP_female_abnormal_train,
        AP_male_normal_train,
        AP_male_abnormal_train
    ],
    axis=0,
    ignore_index=True
)


In [204]:
AP_val = pd.concat(
    [
        AP_female_normal_val,
        AP_female_abnormal_val,
        AP_male_normal_val,
        AP_male_abnormal_val
    ],
    axis=0,
    ignore_index=True
)


In [205]:
AP_test = pd.concat(
    [
        AP_female_normal_test,
        AP_female_abnormal_test,
        AP_male_normal_test,
        AP_male_abnormal_test
    ],
    axis=0,
    ignore_index=True
)


In [206]:
len(AP_train)

14836

In [207]:
# len(AP_train[AP_train['Sex']=='Female']), len(AP_train[AP_train['Sex']=='Male'])
len(AP_val[AP_val['Sex']=='Female']), len(AP_val[AP_val['Sex']=='Male'])
# len(AP_test[AP_test['Sex']=='Female']), len(AP_test[AP_test['Sex']=='Female'])

(1589, 1590)

### frontal PA analysis

In [38]:
df_frontal = df_filtered[df_filtered['Frontal/Lateral']=='Frontal']
df_PA = df_frontal[df_frontal['AP/PA']=='PA']
df_PA

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
5,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Frontal,PA,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
7,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Frontal,PA,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0
16,CheXpert-v1.0-small/train/patient00009/study1/...,Male,76,Frontal,PA,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
18,CheXpert-v1.0-small/train/patient00010/study1/...,Female,50,Frontal,PA,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
32,CheXpert-v1.0-small/train/patient00012/study3/...,Female,61,Frontal,PA,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163679,CheXpert-v1.0-small/train/patient41739/study1/...,Female,62,Frontal,PA,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0
173350,CheXpert-v1.0-small/train/patient44899/study3/...,Female,90,Frontal,PA,NaN,NaN,1.0,1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN
187423,CheXpert-v1.0-small/train/patient51052/study1/...,Male,58,Frontal,PA,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
189937,CheXpert-v1.0-small/train/patient52592/study2/...,Female,32,Frontal,PA,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0


In [39]:
len(df_PA[df_PA['Sex']=='Male']), len(df_PA[df_PA['Sex']=='Female'])

(16610, 8718)

In [58]:
# Normal
PA_normal = df_PA[df_PA['No Finding']==1]

# Abnormal
PA_abnormal = df_PA[df_PA['No Finding']!=1]

In [172]:
PA_normal_female = PA_normal[PA_normal['Sex']=='Female']
PA_normal_male = PA_normal[PA_normal['Sex']=='Male']

PA_abnormal_female = PA_abnormal[PA_abnormal['Sex']=='Female']
PA_abnormal_male = PA_abnormal[PA_abnormal['Sex']=='Male']

In [177]:
RANDOM_STATE = 42

# Shuffle the dataframe
PA_chexpert_female_normal_shuffled = PA_normal_female.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 1388
val_size = 298
test_size = 298

# Create splits
PA_chexpert_female_normal_train = PA_chexpert_female_normal_shuffled.iloc[:train_size]
PA_chexpert_female_normal_val   = PA_chexpert_female_normal_shuffled.iloc[train_size:train_size + val_size]
PA_chexpert_female_normal_test  = PA_chexpert_female_normal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [174]:
RANDOM_STATE = 42

# Shuffle the dataframe
PA_chexpert_male_normal_shuffled = PA_normal_male.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 2380
val_size = 510
test_size = 510

# Create splits
PA_chexpert_male_normal_train = PA_chexpert_male_normal_shuffled.iloc[:train_size]
PA_chexpert_male_normal_val   = PA_chexpert_male_normal_shuffled.iloc[train_size:train_size + val_size]
PA_chexpert_male_normal_test  = PA_chexpert_male_normal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [175]:
RANDOM_STATE = 42

# Shuffle the dataframe
PA_chexpert_female_abnormal_shuffled = PA_abnormal_female.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 2380
val_size = 510
test_size = 510

# Create splits
PA_chexpert_female_abnormal_train = PA_chexpert_female_abnormal_shuffled.iloc[:train_size]
PA_chexpert_female_abnormal_val   = PA_chexpert_female_abnormal_shuffled.iloc[train_size:train_size + val_size]
PA_chexpert_female_abnormal_test  = PA_chexpert_female_abnormal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [176]:
RANDOM_STATE = 42

# Shuffle the dataframe
PA_chexpert_male_abnormal_shuffled = PA_abnormal_male.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 2380
val_size = 510
test_size = 510

# Create splits
PA_chexpert_male_abnormal_train = PA_chexpert_male_abnormal_shuffled.iloc[:train_size]
PA_chexpert_male_abnormal_val   = PA_chexpert_male_abnormal_shuffled.iloc[train_size:train_size + val_size]
PA_chexpert_male_abnormal_test  = PA_chexpert_male_abnormal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [178]:
import pandas as pd

chexpert_PA_train = pd.concat(
    [
        PA_chexpert_female_normal_train,
        PA_chexpert_female_abnormal_train,
        PA_chexpert_male_normal_train,
        PA_chexpert_male_abnormal_train
    ],
    axis=0,
    ignore_index=True
)

In [179]:
import pandas as pd

chexpert_PA_val = pd.concat(
    [
        PA_chexpert_female_normal_val,
        PA_chexpert_female_abnormal_val,
        PA_chexpert_male_normal_val,
        PA_chexpert_male_abnormal_val
    ],
    axis=0,
    ignore_index=True
)

In [180]:
import pandas as pd

chexpert_PA_test = pd.concat(
    [
        PA_chexpert_female_normal_test,
        PA_chexpert_female_abnormal_test,
        PA_chexpert_male_normal_test,
        PA_chexpert_male_abnormal_test
    ],
    axis=0,
    ignore_index=True
)

# vinbig Data

In [301]:
import pandas as pd
csv_path= '/home/jupyter-nafisha/X-ray-covariates/CSVs/vinbig_with_sex.csv'
data = pd.read_csv(csv_path)

In [304]:
data= data[['image_id', 'class_name', 'sex']]
data

,image_id,class_name,sex
0,50a418190bc3fb1ef1633bf9678929b3,No finding,M
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,F
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,F
4,063319de25ce7edb9b1c6b8881290140,No finding,F
...,...,...,...
67909,936fd5cff1c058d39817a08f58b72cae,No finding,NaN
67910,ca7e72954550eeb610fe22bf0244b7fa,No finding,O
67911,aa17d5312a0fb4a2939436abca7f9579,No finding,O
67912,4b56bc6d22b192f075f13231419dfcc8,Cardiomegaly,F


In [305]:
data = (
    data
    .groupby('image_id', as_index=False)
    .agg({
        'class_name': lambda x: ', '.join(sorted(set(
            cls.strip() for cls in x if pd.notna(cls) and cls != ''
        ))),
        'sex': 'first',
    })
)

In [306]:
data

,image_id,class_name,sex
0,000434271f63a053c4128a0ba6352c7f,No finding,O
1,00053190460d56c53cc3e57321387478,No finding,O
2,0005e8e3701dfb1dd93d53e2ff537b6e,"Consolidation, Infiltration, Lung Opacity, Nod...",F
3,0006e0a85696f6bb578e84fafa9a5607,No finding,F
4,0007d316f756b3fa0baea2ff514ce945,"Aortic enlargement, Cardiomegaly, ILD, Pleural...",M
...,...,...,...
14995,ffe6f9fe648a7ec29a50feb92d6c15a4,"Aortic enlargement, Cardiomegaly, Other lesion",F
14996,ffea246f04196af602c7dc123e5e48fc,No finding,O
14997,ffeffc54594debf3716d6fcd2402a99f,Aortic enlargement,M
14998,fff0f82159f9083f3dd1f8967fc54f6a,No finding,O


In [303]:
len(np.unique(data['image_id']))

15000

In [307]:
data['label'] = data['class_name'].apply(
    lambda x: 'Normal' if x == 'No finding' else 'Abnormal'
)

In [308]:
data['image_id'] = data['image_id'].astype(str) + '.dicom'

In [309]:
data

,image_id,class_name,sex,label
0,000434271f63a053c4128a0ba6352c7f.dicom,No finding,O,Normal
1,00053190460d56c53cc3e57321387478.dicom,No finding,O,Normal
2,0005e8e3701dfb1dd93d53e2ff537b6e.dicom,"Consolidation, Infiltration, Lung Opacity, Nod...",F,Abnormal
3,0006e0a85696f6bb578e84fafa9a5607.dicom,No finding,F,Normal
4,0007d316f756b3fa0baea2ff514ce945.dicom,"Aortic enlargement, Cardiomegaly, ILD, Pleural...",M,Abnormal
...,...,...,...,...
14995,ffe6f9fe648a7ec29a50feb92d6c15a4.dicom,"Aortic enlargement, Cardiomegaly, Other lesion",F,Abnormal
14996,ffea246f04196af602c7dc123e5e48fc.dicom,No finding,O,Normal
14997,ffeffc54594debf3716d6fcd2402a99f.dicom,Aortic enlargement,M,Abnormal
14998,fff0f82159f9083f3dd1f8967fc54f6a.dicom,No finding,O,Normal


In [310]:
data_new= data[['image_id', 'sex', 'class_name', 'label']]

In [311]:
data_new

,image_id,sex,class_name,label
0,000434271f63a053c4128a0ba6352c7f.dicom,O,No finding,Normal
1,00053190460d56c53cc3e57321387478.dicom,O,No finding,Normal
2,0005e8e3701dfb1dd93d53e2ff537b6e.dicom,F,"Consolidation, Infiltration, Lung Opacity, Nod...",Abnormal
3,0006e0a85696f6bb578e84fafa9a5607.dicom,F,No finding,Normal
4,0007d316f756b3fa0baea2ff514ce945.dicom,M,"Aortic enlargement, Cardiomegaly, ILD, Pleural...",Abnormal
...,...,...,...,...
14995,ffe6f9fe648a7ec29a50feb92d6c15a4.dicom,F,"Aortic enlargement, Cardiomegaly, Other lesion",Abnormal
14996,ffea246f04196af602c7dc123e5e48fc.dicom,O,No finding,Normal
14997,ffeffc54594debf3716d6fcd2402a99f.dicom,M,Aortic enlargement,Abnormal
14998,fff0f82159f9083f3dd1f8967fc54f6a.dicom,O,No finding,Normal


In [312]:
female_data= data_new[data_new['sex']=='F']
male_data= data_new[data_new['sex']=='M']

In [314]:
# data_new[data_new['sex']=='O']

In [317]:
female_normal = female_data[female_data['label']=='Normal']
female_abnormal = female_data[female_data['label']=='Abnormal']

male_normal = male_data[male_data['label']=='Normal']
male_abnormal = male_data[male_data['label']=='Abnormal']

In [318]:
len(female_normal), len(female_abnormal), len(male_normal), len(male_abnormal)

(1578, 1936, 1902, 1938)

In [319]:
RANDOM_STATE = 42

# Shuffle the dataframe
female_normal_shuffled = female_normal.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 1104
val_size = 237
test_size = 237

# Create splits
female_normal_train = female_normal_shuffled.iloc[:train_size]
female_normal_val   = female_normal_shuffled.iloc[train_size:train_size + val_size]
female_normal_test  = female_normal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [320]:
RANDOM_STATE = 42

# Shuffle the dataframe
female_abnormal_shuffled = female_abnormal.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 1120
val_size = 240
test_size = 240

# Create splits
female_abnormal_train = female_abnormal_shuffled.iloc[:train_size]
female_abnormal_val   = female_abnormal_shuffled.iloc[train_size:train_size + val_size]
female_abnormal_test  = female_abnormal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [321]:
RANDOM_STATE = 42

# Shuffle the dataframe
male_normal_shuffled = male_normal.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 1120
val_size = 240
test_size = 240

# Create splits
male_normal_train = male_normal_shuffled.iloc[:train_size]
male_normal_val   = male_normal_shuffled.iloc[train_size:train_size + val_size]
male_normal_test  = male_normal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [322]:
RANDOM_STATE = 42

# Shuffle the dataframe
male_abnormal_shuffled = male_abnormal.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

# Split sizes
train_size = 1120
val_size = 240
test_size = 240

# Create splits
male_abnormal_train = male_abnormal_shuffled.iloc[:train_size]
male_abnormal_val   = male_abnormal_shuffled.iloc[train_size:train_size + val_size]
male_abnormal_test  = male_abnormal_shuffled.iloc[train_size + val_size:train_size + val_size + test_size]

In [323]:
import pandas as pd

vinbig_train = pd.concat(
    [
        female_normal_train,
        female_abnormal_train,
        male_normal_train,
        male_abnormal_train
    ],
    axis=0,
    ignore_index=True
)

In [324]:
import pandas as pd

vinbig_val = pd.concat(
    [
        female_normal_val,
        female_abnormal_val,
        male_normal_val,
        male_abnormal_val
    ],
    axis=0,
    ignore_index=True
)

In [325]:
import pandas as pd

vinbig_test = pd.concat(
    [
        female_normal_test,
        female_abnormal_test,
        male_normal_test,
        male_abnormal_test
    ],
    axis=0,
    ignore_index=True
)

In [326]:
len(vinbig_train[vinbig_train['label']=='Normal']), len(vinbig_train[vinbig_train['label']=='Abnormal'])

(2224, 2240)

In [327]:
len(vinbig_train[vinbig_train['sex']=='M']), len(vinbig_train[vinbig_train['sex']=='F'])

(2240, 2224)

# concatenation

In [222]:
import numpy as np

chexpert_PA_test['label'] = np.where(
    chexpert_PA_test['No Finding'] == 1,
    'Normal',
    'Abnormal'
)
chexpert_PA_test['Path']= chexpert_PA_test['Path'].str.replace('CheXpert-v1.0-small/train/', '')

disease_cols = [
    'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
    'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
    'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture'
]

chexpert_PA_test['class_name'] = (
    chexpert_PA_test[disease_cols]
    .eq(1)
    .dot(pd.Index(disease_cols) + ', ')
    .str.rstrip(', ')
)

chexpert_PA_test = chexpert_PA_test[['Path', 'Sex', 'class_name', 'label']].rename(
    columns={
        'Path': 'image_id',
        'Sex': 'sex'
    }
)

In [243]:
AP_train['orientation'] = 'AP'
AP_test['orientation'] = 'AP'
AP_val['orientation'] = 'AP'

In [328]:
vinbig_train['orientation'] = 'PA'
vinbig_test['orientation'] = 'PA'
vinbig_val['orientation'] = 'PA'

In [246]:
lateral_train['orientation'] = 'lateral'
lateral_val['orientation'] = 'lateral'
lateral_test['orientation'] = 'lateral'

In [247]:
chexpert_PA_train['orientation'] = 'PA'
chexpert_PA_val['orientation'] = 'PA'
chexpert_PA_test['orientation'] = 'PA'

In [358]:
vinbig_val

,image_id,sex,class_name,label,orientation
0,01a93e09220daa5b0316d426b701c76c.dicom,F,No finding,Normal,PA
1,415da2e8a896b23c97d42cb3ba649467.dicom,F,No finding,Normal,PA
2,2da397f355747dff221c86d9d6eeed87.dicom,F,No finding,Normal,PA
3,abb9ba3432ec27c655d07939da8322fc.dicom,F,No finding,Normal,PA
4,6ac75953da6e3587b8bf97673ee7049f.dicom,F,No finding,Normal,PA
...,...,...,...,...,...
952,247af0fcdbe53ade2def3f79e4eb9345.dicom,M,"Aortic enlargement, Cardiomegaly, Lung Opacity...",Abnormal,PA
953,8d1c24466cec1a18b96989ced1f5422d.dicom,M,"Aortic enlargement, Cardiomegaly, Pleural thic...",Abnormal,PA
954,9c6b55a1b4fe76943377f346658b5da5.dicom,M,"Aortic enlargement, Cardiomegaly",Abnormal,PA
955,def03f80d3b78a52a6f9af13565343ff.dicom,M,"Aortic enlargement, ILD, Lung Opacity, Other l...",Abnormal,PA


In [330]:
train =  pd.concat(
    [
        vinbig_train,
        AP_train,
        lateral_train,
        chexpert_PA_train
    ],
    axis=0,
    ignore_index=True
)

In [331]:
len(vinbig_train), len(vinbig_test), len(vinbig_val)

(4464, 957, 957)

In [332]:
val =  pd.concat(
    [
        vinbig_val,
        AP_val,
        lateral_val,
        chexpert_PA_val
    ],
    axis=0,
    ignore_index=True
)

In [333]:
test =  pd.concat(
    [
        vinbig_test,
        AP_test,
        lateral_test,
        chexpert_PA_test
    ],
    axis=0,
    ignore_index=True
)

In [362]:
import numpy as np

train['image_id'] = np.where(
    train['image_id'].str.endswith('.dicom'),
    'vinbig/' + train['image_id'],
    np.where(
        train['image_id'].str.endswith('.jpg'),
        'chexpert/' + train['image_id'],
        train['image_id']  # unchanged if neither
    )
)
val['image_id'] = np.where(
    val['image_id'].str.endswith('.dicom'),
    'vinbig/' + val['image_id'],
    np.where(
        val['image_id'].str.endswith('.jpg'),
        'chexpert/' + val['image_id'],
        val['image_id']  # unchanged if neither
    )
)
test['image_id'] = np.where(
    test['image_id'].str.endswith('.dicom'),
    'vinbig/' + test['image_id'],
    np.where(
        test['image_id'].str.endswith('.jpg'),
        'chexpert/' + test['image_id'],
        test['image_id']  # unchanged if neither
    )
)

In [380]:
val

,image_id,sex,class_name,label,orientation
0,vinbig/01a93e09220daa5b0316d426b701c76c.dicom,F,No finding,Normal,PA
1,vinbig/415da2e8a896b23c97d42cb3ba649467.dicom,F,No finding,Normal,PA
2,vinbig/2da397f355747dff221c86d9d6eeed87.dicom,F,No finding,Normal,PA
3,vinbig/abb9ba3432ec27c655d07939da8322fc.dicom,F,No finding,Normal,PA
4,vinbig/6ac75953da6e3587b8bf97673ee7049f.dicom,F,No finding,Normal,PA
...,...,...,...,...,...
7157,chexpert/patient21412/study4/view1_frontal.jpg,M,"Cardiomegaly, Pleural Effusion",Abnormal,PA
7158,chexpert/patient24394/study1/view1_frontal.jpg,M,"Lung Opacity, Edema, Pneumonia, Atelectasis, P...",Abnormal,PA
7159,chexpert/patient29068/study1/view1_frontal.jpg,M,Pneumothorax,Abnormal,PA
7160,chexpert/patient12775/study3/view1_frontal.jpg,M,"Lung Opacity, Pleural Effusion",Abnormal,PA


In [335]:
val['sex'] = (
    val['sex']
    .str.strip()
    .str.lower()
    .map({
        'female': 'F',
        'f': 'F',
        'male': 'M',
        'm': 'M'
    })
)
test['sex'] = (
    test['sex']
    .str.strip()
    .str.lower()
    .map({
        'female': 'F',
        'f': 'F',
        'male': 'M',
        'm': 'M'
    })
)
train['sex'] = (
    train['sex']
    .str.strip()
    .str.lower()
    .map({
        'female': 'F',
        'f': 'F',
        'male': 'M',
        'm': 'M'
    })
)

In [364]:
train
test
val

,image_id,sex,class_name,label,orientation
0,vinbig/01a93e09220daa5b0316d426b701c76c.dicom,F,No finding,Normal,PA
1,vinbig/415da2e8a896b23c97d42cb3ba649467.dicom,F,No finding,Normal,PA
2,vinbig/2da397f355747dff221c86d9d6eeed87.dicom,F,No finding,Normal,PA
3,vinbig/abb9ba3432ec27c655d07939da8322fc.dicom,F,No finding,Normal,PA
4,vinbig/6ac75953da6e3587b8bf97673ee7049f.dicom,F,No finding,Normal,PA
...,...,...,...,...,...
7157,chexpert/patient21412/study4/view1_frontal.jpg,M,"Cardiomegaly, Pleural Effusion",Abnormal,PA
7158,chexpert/patient24394/study1/view1_frontal.jpg,M,"Lung Opacity, Edema, Pneumonia, Atelectasis, P...",Abnormal,PA
7159,chexpert/patient29068/study1/view1_frontal.jpg,M,Pneumothorax,Abnormal,PA
7160,chexpert/patient12775/study3/view1_frontal.jpg,M,"Lung Opacity, Pleural Effusion",Abnormal,PA


In [365]:
print(len(train[train['sex']=='M']), len(train[train['sex']=='F']))
print(len(train[train['label']=='Normal']), len(train[train['label']=='Abnormal']))

17220 16202
16202 17220


In [366]:
print(len(test[test['sex']=='M']), len(test[test['sex']=='F']))
print(len(test[test['label']=='Normal']), len(test[test['label']=='Abnormal']))

3690 3472
3472 3690


In [367]:
print(len(val[val['sex']=='M']), len(val[val['sex']=='F']))
print(len(val[val['label']=='Normal']), len(val[val['label']=='Abnormal']))

3690 3472
3472 3690


In [368]:
len(train[(train['sex'] == 'M') & (train['label'] == 'Abnormal')]), len(train[(train['sex'] == 'M') & (train['label'] == 'Normal')]), len(train[(train['sex'] == 'F') & (train['label'] == 'Abnormal')]), len(train[(train['sex'] == 'F') & (train['label'] == 'Normal')])

(8610, 8610, 8610, 7592)

In [369]:
len(test[(test['sex'] == 'M') & (test['label'] == 'Abnormal')]), len(test[(test['sex'] == 'M') & (test['label'] == 'Normal')]), len(test[(test['sex'] == 'F') & (test['label'] == 'Abnormal')]), len(test[(test['sex'] == 'F') & (test['label'] == 'Normal')])

(1845, 1845, 1845, 1627)

In [370]:
len(val[(val['sex'] == 'M') & (val['label'] == 'Abnormal')]), len(val[(val['sex'] == 'M') & (val['label'] == 'Normal')]), len(val[(val['sex'] == 'F') & (val['label'] == 'Abnormal')]), len(val[(val['sex'] == 'F') & (val['label'] == 'Normal')])

(1845, 1845, 1845, 1627)

In [371]:
len(train[train['orientation'] == 'lateral']), len(train[train['orientation'] == 'PA']), len(train[train['orientation'] == 'AP'])

(5594, 12992, 14836)

In [372]:
len(test[test['orientation'] == 'lateral']), len(test[test['orientation'] == 'PA']), len(test[test['orientation'] == 'AP'])

(1198, 2785, 3179)

In [373]:
len(val[val['orientation'] == 'lateral']), len(val[val['orientation'] == 'PA']), len(val[val['orientation'] == 'AP'])

(1198, 2785, 3179)

In [374]:
# 5594+ 18982+ 14836
1198+ 4068+ 3179

8445

In [375]:
train

,image_id,sex,class_name,label,orientation
0,vinbig/c62cdee56fb21cc3554bdecb019539a2.dicom,F,No finding,Normal,PA
1,vinbig/7d99044b9f2dc42f674600c8139f0652.dicom,F,No finding,Normal,PA
2,vinbig/bb3616e0e07167fe64e8f90a3886a8a3.dicom,F,No finding,Normal,PA
3,vinbig/55254db127c8122d04686b5b0e7f1ae2.dicom,F,No finding,Normal,PA
4,vinbig/537b28679b3f9de91401ba50f49faf76.dicom,F,No finding,Normal,PA
...,...,...,...,...,...
33417,chexpert/patient24953/study1/view1_frontal.jpg,M,"Cardiomegaly, Pleural Effusion",Abnormal,PA
33418,chexpert/patient32776/study8/view1_frontal.jpg,M,"Lung Opacity, Pneumothorax, Pleural Effusion",Abnormal,PA
33419,chexpert/patient33810/study1/view1_frontal.jpg,M,Lung Opacity,Abnormal,PA
33420,chexpert/patient21769/study21/view1_frontal.jpg,M,"Enlarged Cardiomediastinum, Atelectasis, Pleur...",Abnormal,PA


In [387]:
train['image_id'] = train['image_id'].str.replace('.dicom', '.jpg')
test['image_id'] = test['image_id'].str.replace('.dicom', '.jpg')
val['image_id'] = val['image_id'].str.replace('.dicom', '.jpg')

In [390]:
train['image_id'] = train['image_id'].str.replace(
    r'(?<!\.)jpg$', '.jpg',
    regex=True
)

test['image_id'] = test['image_id'].str.replace(
    r'(?<!\.)jpg$', '.jpg',
    regex=True
)

val['image_id'] = val['image_id'].str.replace(
    r'(?<!\.)jpg$', '.jpg',
    regex=True
)

In [392]:
train.to_csv('/home/jupyter-nafisha/X-ray-covariates/CSVs/train.csv', index= False)
val.to_csv('/home/jupyter-nafisha/X-ray-covariates/CSVs/val.csv', index= False)
test.to_csv('/home/jupyter-nafisha/X-ray-covariates/CSVs/test.csv', index= False)

In [393]:
train

,image_id,sex,class_name,label,orientation
0,vinbig/c62cdee56fb21cc3554bdecb019539a2.jpg,F,No finding,Normal,PA
1,vinbig/7d99044b9f2dc42f674600c8139f0652.jpg,F,No finding,Normal,PA
2,vinbig/bb3616e0e07167fe64e8f90a3886a8a3.jpg,F,No finding,Normal,PA
3,vinbig/55254db127c8122d04686b5b0e7f1ae2.jpg,F,No finding,Normal,PA
4,vinbig/537b28679b3f9de91401ba50f49faf76.jpg,F,No finding,Normal,PA
...,...,...,...,...,...
33417,chexpert/patient24953/study1/view1_frontal.jpg,M,"Cardiomegaly, Pleural Effusion",Abnormal,PA
33418,chexpert/patient32776/study8/view1_frontal.jpg,M,"Lung Opacity, Pneumothorax, Pleural Effusion",Abnormal,PA
33419,chexpert/patient33810/study1/view1_frontal.jpg,M,Lung Opacity,Abnormal,PA
33420,chexpert/patient21769/study21/view1_frontal.jpg,M,"Enlarged Cardiomediastinum, Atelectasis, Pleur...",Abnormal,PA


In [394]:
combined= pd.concat(
    [
        train,
        test,
        val
    ],
    axis=0,
    ignore_index=True
)

In [395]:
combined.to_csv('/home/jupyter-nafisha/X-ray-covariates/CSVs/combined.csv', index= False)

In [386]:
len(AP_train) + len(lateral_train) + len(chexpert_PA_train) + len(AP_val) + len(lateral_val) + len(chexpert_PA_val) + len(AP_test) + len(lateral_test) + len(chexpert_PA_test) 

41368

In [352]:
len(combined)

47746

In [396]:
directory = '/home/common/data_v3'
count =0 

for _, row in combined.iterrows():
    path= os.path.join(directory, row['image_id'])
    if os.path.exists(path):
        count+=1
count

47746